<a href="https://colab.research.google.com/github/MLMario/mariogj1987/blob/main/A_Clear_Primer_on_Synthetic_Controls_Part_1_%5BWIP%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

The Synthetic Control Method (SCM) is a statistical methodology used primarily for evaluating the impact of interventions, particularly in the realms of economics, policy evaluation, and social sciences. This method provides a data-driven way to construct an 'imaginary' or synthetic control unit that closely approximates the treated unit before the intervention, this allows us to then ask the question: In the absence of treatment, what would have been the results for a given outcome variable?

### **But why should you care as a data scientist?**

Understanding and implementing the Synthetic Control Method is crucial for data scientists in the tech industry, particularly those involved in experimentation and product development. In numerous situations, deploying conventional A/B testing is impractical or problematic, due to inherent complexities like network effects, externalities, or ethical constraints. This is where the prowess of the Synthetic Control Method shines. It potentially enables the design of quasi-experiments where a synthetic unit, constructed through a convex combination of untreated units, serves as the control, and the treatment unit is an average of treated units that are representative of broader group. In short, this is a great option for data scientists to estimate causal impact in constrained environments.


# A real-world application

[In the study by Abadie and Gardeazabal (2003)](https://economics.mit.edu/sites/default/files/publications/The%20Economic%20Costs%20of%20Conflict.pdf), the Synthetic Control Method (SCM) was employed to assess the economic impacts of the conflict in the Basque Country, linked primarily to the terrorist activities of the ETA group.

The authors constructed a synthetic control, utilizing weighted combination of other Spanish regions as potential controls, to represent what the economic trajectory of the Basque Country might have been in the absence of conflict.
The study focused on GDP per capita as the outcome variable and used several economic and demographic characteristics as predictor variables, succesfully finding a 'Synthethic' Pre-Terrorism (the "Intervetion") Basque Country that closely resembles both in GDP per capita (Output variables) and demographic characteristics (Predictor variables)

<div>
<img src="https://drive.google.com/uc?export=view&id=1kZhCAZvAoFNBj4f6rdEzzQeguePtZAKF" width="650"/>
</div>

The results indicated a substantial and persistent divergence in GDP per capita between the Basque Country and its synthetic counterpart, post the onset of terrorism in the 1970s. By 1998, it was estimated that the GDP per capita in the Basque Country was about 10% lower than it would have been without the terrorist conflict.

<div>
<img src="https://drive.google.com/uc?export=view&id=1G9wVXXWUfESvLyHbZS3NX498IuO-rf8G" width="650"/>
</div>


This study underscored the utility of SCM in quantifying the economic repercussions of conflicts, offering empirical insights into the prolonged economic damages stemming from political unrest and terrorism, while also highlighting the need for cautious interpretation due to potential limitations and unobserved confounders.

# The Objective

Mathemathicaly, we are trying to understand what is the average treatment effect for a given outcome $Y$ by comparing:

$ATE = Y^{(1)} _ {t, post} - Y^{(0)} _ {t, post}$

Where the superscript denotes whether the treatment unit $t$ recieved treatment (1) or not (0).

The challenge lies in the fact that typically, the counterfactual outcome for treated units remains unobserved, meaning, we lack insights into what would have occurred to them in the absence of treatment.

For the outcome variable we basically observe 4 types of scenarios given a treatment and control units:

$$
Y =
\begin{bmatrix}
Y^{(1)} _ {t, post} \ & Y^{(0)} _ {c, post} \newline
Y^{(0)} _ {t, pre} \ & Y^{(0)} _ {c, pre}
\end{bmatrix}
$$

The good news, is that if we have access to similar control units that have not recieve a treatment, we can, as mentioned above, use them to construct a synthetic control that basically estimates $Y^{(0)} _ {t, post}$.


# The Process

Assuming you have familiarity with basic matrix algebra and algorithms, I will explain the method using a simplified SCM scenario without incorporating time dimensionality and explaining how to get a synthetic control step by step.

##Step 1: Define the Problem and Assemble Data

Identify the treated unit (the entity undergoing intervention) and potential control units (entities without intervention). Accumulate outcome and predictor variables for these units. Predictor variables are essential covariates believed to influence the outcome.

**Note:** Take notice that the treatment units here are taken as a given, this an assumption that will be present in all extensions of this method up to the case where we actually want to design an experiment

##Step 2: Calculate an initial set of weights $W$

The crux is to find a vector of weights, $W$ , ensuring that the weighted combination of control units closely resembles the treated unit regarding predictor and outcome variables *before the intervention*. To get to this, we will start by first decreasing the distance between predictor variables.

For two entities, treated and control, with predictor variables denoted by
$X$ matrices, we formulate the objective as:

$ W^*(V) = min_W  (X_{t,pre} - X_{c,pre}*W)^T*V*(X_{t,pre} - X_{c,pre}*W)$

$Where:$

$X_{t,pre}$ is the matrix of predictor variables for the treated unit.

$X_{c,pre}$  is the matrix of predictor variables for the control units.

$V$  is a diagonal matrix determining the importance of each predictor variable .

$W_i \ge 0$

$\sum_{W_i\in J} = 1$

The contraints on the weights are there to help avoid overfitting and to maintain interpretabiilty, for example, Country A = 2*Countries B - 1*Country C doesnt quite make sense in that context. Having said that, this restriction can be relaxed depending on the Synthethic control Method we are using.

## Step 3: Assesing the fit

After obtaining the optimal weights and constructing the synthetic control, you assess how well the synthetic control replicates the treated unit in the pre-intervention period by comparing their outcomes.

$Y_{syn,pre} = Y_{c,pre}*W^*$


$Outcome Error = Y_{t,pre} - Y_{syn,pre}$

If the fit is good, we can stop here, if not, you can move on to step 4

##Step 4: Selecting $V$

The minimization of this distance is equivalent to minimizing the distance between the control and treatment unit pre-treatment, but in itself  treated unit's outcome variable $Y$ during the pre-treatment period.

To get the synthetic control to match the treated unit in terms of both predictor variables and the outcome variable during the pre-treatment period, we take an iterative approach

*Here's the procedure:*

1) Start by finding weights $W$ that minimize the aforementioned distance, so the synthetic control matches the predictor variables of the treated unit.
Check how well the synthetic control matches the treated unit's outcome variable $Y_{t,pre}$ during the pre-treatment period.

2) If the match is not within an acceptable error range, adjust the diagonal matrix $V$ to give more weight to predictor variables that seem to be influential in improving the match in the outcome variable. Then, recompute $W$.

3) Repeat the above steps until you achieve a satisfactory match in the outcome variable $Y_{t,pre}$ during the pre-treatment period.

So, while the distance formula does not explicitly contain $Y$, the iterative procedure ensures that the synthetic control matches $Y$ closely during the pre-treatment period by adjusting the importance of the predictor variables (through $V$) based on how they impact the match in $Y$


# A simple example

We are going to generate a synthetic control to explain better the procedure explained above. To start, it is convenient to make sure we understand what the data looks like. In general, we should be able count with balanced panel data (same amount of observed periods for every unit)

## Creating the dataset

- For this example, I'm creating a very simple dataset where the time period is simplified as pre and post periods. Once we move away from this formulation the actual math becomes a bit more complicated, but the forumulation stays the same.

In [ ]:
import pandas as pd
import numpy as np

# Correcting the DataFrame construction
data = {
    'unit': ['treated', 'treated', 'control_1', 'control_1', 'control_2', 'control_2', 'control_3', 'control_3'],
    'time': ['pre', 'post', 'pre', 'post', 'pre', 'post', 'pre', 'post'],
    'outcome': [5, 15, 4, 10, 6, 12, 3, 6.5],  # Example outcome variable
    'predictor_1': [2, 4, 4, 6, 1.5,3.5, 4, 8],  # Example predictor variable 1
    'predictor_2': [2.5, 6, 4.5, 8.5, 3, 5.8, 2.5, 5],  # Example predictor variable 2
    'predictor_3': [1, 2.1, 2, 3.5, 3, 5.5, 4, 3.3]  # Example predictor variable 3
}

# Convert the dataset into a DataFrame and display it
df = pd.DataFrame(data)
df

,unit,time,outcome,predictor_1,predictor_2,predictor_3
0,treated,pre,5.00,2.00,2.50,1.00
1,treated,post,15.00,4.00,6.00,2.10
2,control_1,pre,4.00,4.00,4.50,2.00
3,control_1,post,10.00,6.00,8.50,3.50
4,control_2,pre,6.00,1.50,3.00,3.00
5,control_2,post,12.00,3.50,5.80,5.50
6,control_3,pre,3.00,4.00,2.50,4.00
7,control_3,post,6.50,8.00,5.00,3.30


- Since the weights are calculated only use pre-treatment data, lets separate the data into pre and post

In [ ]:
# Separate the dataset into pre-treatment and post-treatment DataFrames
df_pre = df[df['time'] == 'pre'].reset_index(drop=True)
df_post = df[df['time'] == 'post'].reset_index(drop=True)

# Display the pre-treatment DataFrame
df_pre

,unit,time,outcome,predictor_1,predictor_2,predictor_3
0,treated,pre,5.00,2.00,2.50,1.00
1,control_1,pre,4.00,4.00,4.50,2.00
2,control_2,pre,6.00,1.50,3.00,3.00
3,control_3,pre,3.00,4.00,2.50,4.00


### Define starting V

- $V$ is $k * k$ matrix, where $k$ is the number of predictors.

In [ ]:
# Define the treated unit and control units
treated_unit = df_pre[df_pre['unit'] == 'treated']
control_units = df_pre[df_pre['unit'] != 'treated']

# Define the initial V matrix as an identity matrix
k = 3  # Number of predictor variables
V = np.identity(k)
V

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

### Find $W^*$

- We now minimize the difference between the predictors in treatment and control.  

In [ ]:
from scipy.optimize import minimize

# Define the objective function to minimize.
# Note that this is simple returning the minimization target stated in Step 2 of the mathematical formulation
def objective_function(W, X0, X1, V):
    diff = X1 - np.dot(X0, W)  # Difference between X1 and weighted sum of X0
    diff = diff.flatten()  #The subtraction X1 - np.dot(X0, W) results in a two-dimensional array with a single row (i.e., a row vector). so it needs to be flatten to 1-D vector
    return np.dot(np.dot(diff.T, V), diff)  # Corrected matrix multiplication

In [ ]:
# define the amount of units in control
J = control_units.shape[0]

#define initial weights

# Initialize the weights equally
initial_weights = [1/J] * J

''' Define the constraints '''

# The weights must sum to one
constraints = (
    {'type': 'eq', 'fun': lambda W: np.sum(W) - 1},
)

# The weights must be between 0 an 1
bounds = [(0, 1) for _ in range(J)]


- Lets take some time to extract the predictor variables for both treatment and control and observe them



In [ ]:
X1 = treated_unit[['predictor_1', 'predictor_2', 'predictor_3']].values  # 1xk vector for treated unit
X0 = control_units[['predictor_1', 'predictor_2', 'predictor_3']].values  # Jxk matrix for control units
Y1 = treated_unit['outcome'].values  # 1x1 vector for treated unit
Y0 = control_units['outcome'].values  # 3x1 vector for control unit

In [ ]:
print(X1)

[[2.  2.5 1. ]]


In [ ]:
print(X0)

[[4.  4.5 2. ]
 [1.5 3.  3. ]
 [4.  2.5 4. ]]



Note that in $X1$ (a.k.a $X_{t,pre}) we have 1 time period, 1 treated unit and 3 predictors, resulting in a 1 * 3 matrix.

$X0$ now has 1 time period 3 control units and 3 predictors, results in a 3 *3 matrix.

If we where to extend this formulation to,let say ten period, $X1$ would now be a 10 * 3 matrix, but $X0$ would be a 10 * 3* * 3 *tensor*, that is a matrix of matrix. This formulation is neccesary because now we want to minimize the distance between vectors, more on this later when we expand the concept.

- now lets actually calculate W


In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)

def calculate_w(objective_function,initial_weights,X0,X1,V,constraints,bounds):

  # Perform the optimization to find the optimal weights W again
  result = minimize(objective_function, initial_weights, args=(X0, X1, V), constraints=constraints, bounds=bounds)

  # Extract the optimal weights and display them along with the unit names
  optimal_weights_scipy = result.x

  # Extract the control unit names
  control_unit_names = control_units['unit'].tolist()


  return optimal_weights_scipy

In [ ]:
W = calculate_w(objective_function,initial_weights,X0,X1,V,constraints,bounds)
pd.DataFrame({'Control Unit': control_unit_names, 'Weight': W})

,Control Unit,Weight
0,control_1,0.00
1,control_2,0.53
2,control_3,0.47


- The weights are essentially assigning all the importance to control_2 and control_3

### Create Synthetic Control and Asses Fit

- Lets start by predict the outome and predictor variables for our synthethic control unit. We can

In [ ]:
# Calculate the synthetic predictors and outcomes for the pre-treatment period
def calculate_syn(X0,Y0,optimal_weights_scipy):
  X_syn = np.dot(X0.T, optimal_weights_scipy)
  Y_syn = np.dot(Y0,optimal_weights_scipy)
  return X_syn, Y_syn


def compare_results(Y_syn, Y1, has_time = False):
  return sum(Y_syn - Y1) / sum(Y1)


In [ ]:

def visualize_results(X_syn,X1,Y_syn, Y1):

  synthetic_control_pre_df = pd.DataFrame(X_syn.reshape(1, -1), columns=['predictor_1', 'predictor_2', 'predictor_3'])
  synthetic_control_pre_df['outcome'] = Y_syn
  synthetic_control_pre_df['unit'] = 'synthetic'
  synthetic_control_pre_df['time'] = 'pre'

  # Combine the synthetic control with the treated unit for comparison
  comparison_pre_df = pd.concat([treated_unit, synthetic_control_pre_df], axis=0)

  # Display the comparison DataFrame for the pre-treatment period
  return comparison_pre_df



In [ ]:

X_syn, Y_syn = calculate_syn(X0,Y0,optimal_weights_scipy)
outcome_diff = compare_results(Y_syn, Y1, has_time = False)

print(f'Relative difference of outcome synthetic vs treated pre-treatment:{outcome_diff:.3f}')

Relative difference of outcome synthetic vs treated pre-treatment:0.200


In [ ]:
visualize_results(X_syn,X1,Y_syn, Y1)

,unit,time,outcome,predictor_1,predictor_2,predictor_3
0,treated,pre,5.00,2.00,2.50,1.00
0,synthetic,pre,6.00,1.50,3.00,3.00


### Iterating over $V$ values to improve fit.

####Steps to Adjust $V$

**1. Define a Criterion:** Choose a criterion to adjust the elements of
$V$. This could be based on domain knowledge or systematic methods like cross-validation.

**2. Update $V$:** Modify the elements of $V$ based on the chosen criterion. For example, if one predictor variable seems more important, increase its corresponding weight in $V$.

**3. Re-optimize $W$:** With the updated
$V$, re-optimize the weights $W$ to minimize the objective function.

**4. Assess the Fit:** Evaluate how well the synthetic control with the new weights approximates the treated unit during the pre-treatment period.

#### Example: Simple Update Rule

One simple way to adjust $V$ is to look at the absolute differences between the treated unit and the synthetic control for each predictor in the pre-treatment period and assign higher weights to predictors with larger differences.

$V_i = 1 /{1- |diff_i|}$

where V_i  is the weight for the $i^{th}$ predictor in the $V$ matrix, and
$|diff_i|$ is the difference between the treated unit and the synthetic control for the i^{th} predictor in the pre-treatment period.

array([[1.5 , 2.5 , 0.75]])

In [ ]:
def adjust_v(V,X1,X_syn):

  # Calculate the absolute differences between the treated unit and synthetic control for each predictor in the pre-treatment period
  differences = np.abs(X1 - X_syn)

  # Update the V matrix based on the simple update rule
  V_adjusted = np.diag(1 / (1 + differences.flatten()))

  # Display the adjusted V matrix
  return V_adjusted

In [ ]:

def optimize_v(V,X1,X0,Y1,Y0,Y_syn,X_syn,W,objective_function,constraints,bounds,max_try = 100):


  X_syn_adjusted = X_syn.copy()
  V_adjusted = V.copy()
  W_adjusted = W.copy()


  # Iterate over the maximum number of tries
  for i in range(max_try):

    # Adjust the V matrix
    V_adjusted = adjust_v(V_adjusted,X1,X_syn_adjusted)

    print(V_adjusted)

    #recalculate weights
    W_adjusted = calculate_w(objective_function,W_adjusted,X0,X1,V_adjusted,constraints,bounds)

    #recalculate synthetic control
    X_syn_adjusted, Y_syn_adjusted = calculate_syn(X0,Y0,W_adjusted)


    outcome_diff = compare_results(Y_syn_adjusted, Y1, has_time = False)
    print(abs(outcome_diff))

    if abs(outcome_diff) < 0.15:
      break

  return X_syn,Y_syn, V_adjusted

In [ ]:
 #recalculate synthetic control
X_syn,Y_syn, V_adjusted = optimize_v(V,X1,X0,Y1,Y0,Y_syn,X_syn,W,objective_function,constraints,bounds,max_try = 1)

[[0.46897208 0.         0.        ]
 [0.         0.85212599 0.        ]
 [0.         0.         0.27375291]]
0.19161861473115777


In [ ]:
abs(outcome_diff)

0.1999999999999961

In [ ]:
abs(-outcome_diff)

0.1999999999999961